In [10]:
import io
import os
import pyaudio
import wave
from google.cloud import speech

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_JSON_FILE" # Set authentication

client = speech.SpeechClient()

# Configure microphone
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

print("Listening... Speak now.")

def recognize_speech():
    frames = []
    for _ in range(0, int(RATE / CHUNK * 5)):  # Record for 5 seconds
        data = stream.read(CHUNK)
        frames.append(data)

    # Save audio file
    wf = wave.open("audio.wav", "wb")
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    # Read and send audio to Google Cloud
    with io.open("audio.wav", "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US"
    )

    response = client.recognize(config=config, audio=audio)

    for result in response.results:
        return result.alternatives[0].transcript  # Return recognized text

    return ""

text = recognize_speech()
print(f"You said: {text}")


Listening... Speak now.
You said: hello


In [1]:
pip install openai

   ---------------------------------------- 0.0/473.5 kB ? eta -:--:--
   --------------- ------------------------ 184.3/473.5 kB 5.6 MB/s eta 0:00:01
   ----------------------------------- ---- 419.8/473.5 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 473.5/473.5 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.7 kB ? eta -:--:--
   --------------------------------------- 204.7/204.7 kB 13.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### YOU WILL NEED TO PAY TO USE THE OPENAI API, UNLESS YOU HAVE FREE TRIAL, IN WHICH CASE CHECK WHICH MODEL IS AVAILABLE AND SWAP THE MODEL ATTR. IN RESPONSE OBJ

In [ ]:
import openai

openai.api_key = "API_KEY"  # Replace with your actual OpenAI API key

client = openai.OpenAI(api_key=openai.api_key)  # Pass the API key explicitly

def get_humorous_response(user_input, humor_level=0.9):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"You are TARS, a witty AI with a humor setting of {humor_level}."},
            {"role": "user", "content": user_input}
        ],
        temperature=humor_level
    )
    return response.choices[0].message.content

# Test
user_input = "Tell me something about black holes."
tars_response = get_humorous_response(user_input)
print(f"TARS: {tars_response}")

In [ ]:
from google.cloud import texttospeech
import simpleaudio as sa

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_JSON_FILE"  # Set authentication

client = texttospeech.TextToSpeechClient()

def speak(text):
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code="en-US", name="en-US-Wavenet-D")
    audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.LINEAR16)

    response = client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)

    with open("response.wav", "wb") as out:
        out.write(response.audio_content)

    # Play the audio
    wave_obj = sa.WaveObject.from_wave_file("response.wav")
    play_obj = wave_obj.play()
    play_obj.wait_done()

speak("Hello! I am TARS. How can I assist you?")


In [3]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [5]:
%cd yolov5

C:\Users\aarna\Downloads\tars-things\yolov5


In [7]:
!pip install -r requirements.txt

In [ ]:
import cv2
import torch
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression
from yolov5.utils.plots import Annotator  # New module for drawing boxes
from yolov5.utils.torch_utils import select_device

# Load YOLO model
device = select_device('cpu')  # Change to 'cuda' if using GPU
model = DetectMultiBackend("yolov5s.pt", device=device)
model.eval()

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    img = torch.from_numpy(frame).permute(2, 0, 1).float() / 255.0
    img = img.unsqueeze(0)

    with torch.no_grad():
        preds = model(img)

    detections = non_max_suppression(preds, 0.5, 0.45)

    for det in detections:
        if len(det):
            annotator = Annotator(frame)
            for *xyxy, conf, cls in det:
                label = f"{model.names[int(cls)]} {conf:.2f}"
                annotator.box_label(xyxy, label)

    cv2.imshow("TARS Vision", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


YOLOv5  v7.0-398-g5cdad892 Python-3.12.4 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
